# Simulation

In this notebook we aim to complete three simulation processes
* the travel time form zone i to zone j start from time k
* the number of travels from zone i to zone j at time k
* find out the correlation of fare on travel distance and travel time

If two trips that start and end in same zone, and begin from same time, we can assume it takes similar times to complete.

## Zone to zone time simulation

In this section we aim to simulate the time it takes for a ride from zone to zone at specific time. This section consists of three parts:
* gather rides that have same pickup and dropoff zones
* apply empirical distribution to simulate PDF
* simulate potential time for rides

### Data import and environment setup

In [1]:
import pandas as pd
import numpy as np
import bisect
import random
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression

# set specific seed for reproduction
random.seed(0)

In [2]:
# import cleaned data from taxi_preprocessing.ipynb
data = pd.read_csv("nyc_data_cleaned.csv")
number_of_zone = len(data["pickup_zone"].unique())
data

,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,total_amount,pickup_hour,pickup_minute,pickup_second,...,dropoff_minute,dropoff_second,pickup_timestamp,dropoff_timestamp,pickup_latitude_count,pickup_longitude_count,dropoff_latitude_count,dropoff_longitude_count,pickup_zone,dropoff_zone
0,1801,5.0,-74.013359,40.714722,-73.972649,40.764694,23.5,18,30,59,...,1,57,66659,68517,2,1,7,5,2,23
1,172,1.0,-73.964958,40.760021,-73.971352,40.751297,7.0,19,19,7,...,22,40,69547,69760,7,6,6,5,24,19
2,260,2.0,-73.960114,40.762142,-73.974434,40.744308,8.5,2,31,6,...,34,46,9066,9286,7,7,5,5,25,15
3,521,2.0,-73.968163,40.802277,-73.984322,40.780106,9.0,12,20,6,...,29,27,44406,44967,11,6,9,4,39,31
4,512,3.0,-73.973289,40.792690,-73.973602,40.763554,10.5,6,9,0,...,18,0,22140,22680,10,5,7,5,36,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347809,907,7.0,-74.005219,40.720135,-73.951187,40.770283,27.0,23,59,49,...,14,56,86389,896,3,2,8,7,5,29
347810,465,2.0,-73.992271,40.729198,-73.975388,40.757710,10.0,23,59,50,...,7,35,86390,455,4,3,6,5,9,19
347811,395,2.0,-73.987450,40.757973,-74.003304,40.750622,8.0,23,59,51,...,6,26,86391,386,6,4,6,2,18,16
347812,948,2.0,-73.993874,40.742134,-73.967232,40.756779,16.2,23,59,58,...,15,46,86398,946,5,3,6,6,13,20


### Time simulation

In [3]:
# setup a 3-dimensional list to store the simulated result
simulated_time = [
    [
        [0] * (60 * 24) for _ in range(number_of_zone)
    ] for _ in range(number_of_zone)
]

`simulated_time[i][j][k]` means the travel time from zone i to zone j at start_time k.

***Below part can be accelarated***

In [4]:
# a helper function to determine the PDF of empirical distribution of travel_time
# and then do simulation
def empirical_simulation(i, j, t, travel_time, average_time):
    
    # if no data is provided, use the average travel time of the day
    if len(travel_time) == 0:
        simulated_time[i][j][t * 60: (t + 1) * 60] = [average_time] * 60
        return
    
    # simulate ecdf on given time for rides
    ecdf = np.cumsum(np.bincount(travel_time)) / len(travel_time)
    interp_func = interp1d(ecdf,
                           np.arange(len(ecdf)),
                           kind='linear',
                           bounds_error=False,
                           fill_value='extrapolate')

    # generate 60 random samples from the fitted distribution
    # so that each trips start from each minute is simulated in this hour
    samples = interp_func(np.random.uniform(low=0, high=1, size=60*24))
    simulated_time[i][j][t * 60: (t + 1) * 60] = list(samples)

In [5]:
for i in range(number_of_zone):
    for j in range(number_of_zone):
        df = data[(data["pickup_zone"] == i)
                  & (data["dropoff_zone"] == j)]
        average_travel_time = df["trip_time_in_secs"].mean()
        
        # t stands for hour information
        for t in range(24):
            
            # df_sub includes only the rides from zone i to zone j during t:t+1 hour
            df_sub = data[(data["pickup_zone"] == i)
                          & (data["dropoff_zone"] == j)
                          & (data["pickup_hour"] == t)]
        
            # get all the travel time for rides
            cur_travel_time = np.array(df_sub["trip_time_in_secs"]).astype(np.int64)
            empirical_simulation(i, j, t, cur_travel_time, average_travel_time)

Therefore, the `simulated_time` is implemented and is easily accessible.

## Zone to zone trip simulation

In this part we are dealing with simulation on the number of trips. It follows similar ideas to the previous part.

In [6]:
# setup a 3-dimensional list to store the simulated result
simulated_trip = [
    [
        [0] * (60 * 24) for _ in range(number_of_zone)
    ] for _ in range(number_of_zone)
]

In [7]:
# a helper function to determine the PDF of empirical distribution of travel_time
# and then do simulation
def trip_simulation(i, j, t, travel_count):
    samples = np.random.multinomial(travel_count, np.ones(60) / 60)
    simulated_trip[i][j][t * 60: (t + 1) * 60] = list(samples)

In [8]:
for i in range(number_of_zone):
    for j in range(number_of_zone):
        for t in range(24):
            
            # df_sub includes only the rides from zone i to zone j during t:t+1 hour
            df_sub = data[(data["pickup_zone"] == i)
                          & (data["dropoff_zone"] == j)
                          & (data["pickup_hour"] == t)]
        
            # get all the travel time for rides
            cur_trip_number = len(df_sub)
            trip_simulation(i, j, t, cur_trip_number)

Therefore, the `simulated_trip` is implemented and is easily accessible.

## Ride fare simulation

In this part we are dealing with simulation of fare for each ride. Conventionally, we believe fare is related to the distances of the ride and the time spent on the ride.

We will fit a linear regression model to find out this correlation. Moreover, we will provide distance information between zones so that price could be simulated for each simulated ride.

We aim to fit the following model:
$$
\begin{aligned}
\text{Fare} = \beta_0 + \beta_1 \text{Distance} + \beta_2 \text{Time} + \varepsilon
\end{aligned}
$$

In [10]:
fare = np.array(data["total_amount"])
distance = np.array(data["trip_distance"]).reshape((-1, 1))
travel_time = np.array(data["trip_time_in_secs"]).reshape((-1, 1))

x = np.concatenate((distance, travel_time), axis=1)

# create a linear regression model and fit it to the data
fare_model = LinearRegression().fit(x, fare)

# calculate R-squared
r_squared = fare_model.score(x, fare)
print("The R squared of the model is: ", r_squared)

The R squared of the model is:  0.8705780665239197


From above, we know that the fitting of this fare model is good. So that we can use the above model for fare simulation.

In [11]:
beta0 = fare_model.intercept_
beta1 = fare_model.coef_[0]
beta2 = fare_model.coef_[1]

The average distances of zone i to zone j can also be calculated.

In [12]:
# create a 2-dimension list to store the average distance between zones.
# in this scenerio we assume dis(i, j) != dis(j, i)
average_distance = [
    [0] * number_of_zone for _ in range(number_of_zone)
]

for i in range(number_of_zone):
    for j in range(number_of_zone):
        df = data[(data["pickup_zone"] == i)
                  & (data["dropoff_zone"] == j)]
        average_distance[i][j] = df["trip_distance"].mean()

We can write another function to calculate the simulated fare

In [13]:
def simulated_fare(distance, time):
    return beta0 + beta1 * distance + beta2 * time

Therefore, we have already completed simulation on trip number, trip time and trip fare.

For example, we can randomly select i, j, k as follows

In [17]:
# zone 13 to zone 18 at time of 17:30
i, j, k = 13, 18, 17 * 60 + 30

print("Simulated trip number: ", simulated_trip[i][j][k])
print("Simulated travel time: ", simulated_time[i][j][k], "seconds")
print("Simulated travel distance", average_distance[i][j], "miles")
print("Simulated fare: ",
      simulated_fare(average_distance[i][j], simulated_time[i][j][k]),
      "dollars")

Simulated trip number:  1
Simulated travel time:  698.2103994224359 seconds
Simulated travel distance 1.0836548223350277 miles
Simulated fare:  10.276778131546862 dollars
